# Initialize
This section is a developer aid; TODO remove later!
*Align with [storyboard Version 48](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=48)*

If you are using this under Time Square and *get any errors* that you think come from this notebook, please tell Steve Pothier (steve.pothier@noirlab.edu).  Include the URL you used to get the error. 

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "2024-11-07"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "6"  # TODO Change to '1' to test with default before push

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", 
    ut.Server.usdf) # TODO try with "usdf" before push (else "summit")

In [4]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 5000  # YAGNI for Auto get more if this isn't enough to get all requested DAYS
response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

# Experimental Features config
exp_tally = "false"  # default: true
exp_parsing = "true"  # default: true
experimental = dict()
experimental["exposure_type_tally"] = exp_tally == "true"
experimental["error_parsing"] = exp_parsing == "true"

In [5]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=True,                # TODO change to False before push
    limit=limit,
    exclude_instruments = []      # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

DBG get_records endpoint='https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports'
DBG get_records qstr: is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241102&max_day_obs=20241108
DBG get_records len(page)=9 len(recs)=0
DBG get_records-2 len(page)=9 len(recs)=9
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr: registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241102&max_day_obs=20241108
DBG get_exposures len(page)=0 len(recs)=0
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr: registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241102&max_day_obs=20241108
DBG get_exposures len(page)=0 len(recs)=0
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr: registry=1&instrument=LSSTComCam&order_b

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")

# Showing data for 2024-11-02 to 2024-11-08 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

# Table of Contents
* [Night Report](#Night-Report-BETA)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac-BETA)
* [Summary plots of whole night](#Summary-plots-of-whole-night-DRAFT)
* [Time Accounting](#Time-Accounting-DRAFT)
* [Jira Tickets](#Jira-Tickets-BETA)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed-DRAFT)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log-DRAFT)
* [Narrative Log](#Narrative-Log-BETA)
* [Developer Only Section](#Developer-Only-Section-REMOVE)

## Night Report <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

### Telescope: AuxTel

```
We had a bumpy start to the beginning of the observation day. During the bump tests, FA 235 failed several times, but we got confirmation that it was safe to raise the mirror. We had repeating issues with the MTHexapod today during its warmup. It failed several times, so we skipped the procedure - see OBS-620. We decided to reset the PXI for the M2 hexapod, and managed to initiate compensation mode. The TMA had some buggy behaviors as well. We tried to save some sky time and slew the TMA to the south well before sunset. Due to soft motion axis faults, and some weird but faultless behaviors (see details in the narrative log), it took around two hours to get the TMA into position. We went to dinner, came back, and prepared to open by sunset.

In addition to that afternoon fun, the ambient temperature went nuts starting around 2 hours before sunset, plummeting and rising again on a crazy roller coaster. 🎢 We opened just after sunset, fooled into thinking it had stabilized, and the M1M3 temperature dropped at an alarming rate of about 0.4 C per 15 minutes. We closed quickly, and turned on the HVAC systems to a 6 degree set point to try and keep M1M3 closer to ambient. Humidity began creeping up around 9:30 - reaching around 70%. We remained closed and continued in dome testing.

As the weather changed frequently, we opened the dome in steps to slowly thermalize the mirror while running the LUT. We decided to fully open around midnight and began on-sky testing. 

AOS night Summary:

Due to weather and temperature, we weren’t able to fully open the dome until ~00:30 local time. We used the time before this to perform T222 - Elevation LUT with closed dome.  Then we started T88 Laser Alignment to determine a good starting point. Using this starting point we started T92 to step though varying the M2 hexapod and camera hexapod dz positions to refine the focus.  First we moved the two hexapod shifts in the same direction and then had them move in different directions.  We found while doing this that we couldn’t return to the position we found in focus.  We determined this was due to the fact that we were not giving uncompensated positions.  We were using the compensated values and effectively applying them twice.  We built a new dashboard with uncompensated positions and then asked to go back to the uncompensated position via a run_command. It worked!  This explains why were weren’t able to reproduce values on previous nights.

With this information we once again tried the parameter march with the two hexapods moving in opposite directions.  We weren’t able to finish in time, and tried T202 which attempted to record the state of the system with the laser tracker.  We started the script and soon realized the system was not in focus. We stopped the script, but in fact the process did not end, and the laser tracker task completed and moved from a M1M3 SMR finally stopping at a Camera SMR.  When we next took an image to check focus, the extra light from the laser on the camera SMR tripped the voltage levels on the ComCom REBs, causing the ComCam to go into fault.  Brian Stalder remotely recovered the ComCam and we finished our tests due to the required closing time.
```

Confluence page: [testCycle/BLOCK-R126](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testCycle/BLOCK-R126)

Telescope Status: 

*Authors: David Sanmartim, Alysha Shugart, Guillem Megias Homar, Chris Walter, Carlos Morales*

...

Telescope Status: TMA parked at El: 90deg (Zenith), Az: -32 with Dome: 328 deg per requested. Locking pins are engaged, OSS ON. PS OFF. All CSCs are set in the states as requested: MTM1M3 DISABLED - lowered and disabled, MTM2 ENABLED in closed loop, MTAOS STANDBY, MTDome DISABLED, MTDomeTrajectory STANDBY, MTMount DISABLED,  MTHexapods in STANDBY w/ cabinets and PXIs powered off, MTPtg ENABLED, MTRotator needs to be back at 0º, in STANDBY w/ cabinets and PXIs powered off. 

*Authors: Kshitija Kelkar, Brian Stalder, Yijung Kang, Sandrine Thomas, Robert Lupton, Chris Walter, Elana Urbach, Josh Meyers, Guillem Megias Homar*

```
The day started with powering up the rotator and the hexapods. The worming up was scheduled for the late afternoon. After checking with Jacques and Brian, we agreed to leave the temperature set point to 11C. Information from Freddy: There is an auxiliary pump close to OSS . Hoses are connected but system is DISABLED (filter case open) and is not related AT ALL with the operation of the telescope. However, the open filter was dripping oil to a unusual rate (we had this information from the beginning of the day, to be aware of). At 2:05pm (CLT) we had a discuttion with Brian and Te-Wei about worming up the hexapods and we decided to do it a bit earlier. While M2Hexapods are warming up, Marina, Gonzalo and I went up to Level 8 for the TMA and Dome Handover. The handover went smoothly. Right after the handover we got confirmation from Petr that we are good to go on HardPoint Breakaway tests (BLOCK-T145), that is started right away. The hardpoint breakaway test was performed on the EUI.
After the hardpoint test we ran the bunmp test and during this time the elevation drives failed and David Jimenez recovered the TMA. After bump test we opened the mirror covers to perform a laser tracker test. In the meantime Gabrielle ran the M2 bump test which triggered some alarms (?) which were resolved quickly. Then, we attempted to perform the laser tracker alignment, but the script ran too quickly. The mirror covers got stuck when we tried to close them, but this issue was resolved by running the command again.

We experienced a minor earthquake near the start of the late shift ~9:15pm.  After the earthquake, MTM1M3, MTMount, and MTPtg went to fault and we recovered them.  After that, we could not solve the MTDome telemetry issues.  After three hours of troubleshooting, we opted to close the dome and perform closed-dome tests, including the TMA Azimuth and Elevation Brake Distance - 2% Settings tests.  We had an issue with the EIB which we resolved by rebooting the encoder hardware in the EUI.  We then proceeded to run the Block T-227 Dynamic Test at 34 deg, but ran into issues when passing the configuration in the move_p2p.py script.  Our solution was to manually add each p2p position to the script queue and run.  We didn't experience any issues with this approach.  After this completed, we transitioned to park the TMA and end the night.  We lowered the mirror.  The only issue we had when ending the night was when we sent MTM1M3 to disabled by CSC.  We cleared the interlock and put MTM1M3 in disabled.
```

Telescope Status: The final state is:
HVAC set point = 11 deg C 
We kept on the FCU with 300 RPM 
MTM1M3 DISABLED - lowered and disabled
MTM2 ENABLED in closed loop
MTAOS STANDBY
MTDome DISABLED
MTDomeTrajectory STANDBY
MTMount DISABLED
MTHexapods in STANDBY w/ cabinets and PXIs powered off
MTPtg ENABLED
MTRotator at 0 deg, in STANDBY w/ cabinets and PXIs powered off


TMA is at the zenith position, Az = -32
OSS Power ON 
Power supply OFF 
Dome position 328 deg
Mirror covers are closed 
Locking pins in lock position 

*Authors: Bruno Quint, Marina Pavlovic, Gonzalo Aravena, Kate Napier, Carlos Morales*

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-11-02**

,(times in UTC),
Moon Rise,2024-11-02 10:13:45.292,
Moon Set,2024-11-01 23:24:38.032,
Moon Illumination,0%,(% illuminated)
Astronomical Twilight (morning),2024-11-02 08:19:39.358,(-18 degrees)
Astronomical Twilight (evening),2024-11-02 00:33:05.276,(-18 degrees)
Solar Midnight,2024-11-02 04:26:22.503,
Nautical Twilight (morning),2024-11-02 08:51:02.265,(-12 degrees)
Nautical Twilight (evening),2024-11-02 00:01:42.202,(-12 degrees)
Civil Twilight (morning),2024-11-02 09:21:16.302,(-6 degrees)
Civil Twilight (evening),2024-11-01 23:31:27.941,(-6 degrees)


## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## Time Accounting <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
TODO:
- Time-loss for Fault and Weather
- SlewTime from TMAEvent (or similar)

SlewTime (and probably others) from EFD are only EXPECTED (topic="lsst.sal.Scheduler.logevent_target"), not ACTUAL.
To get ACTUAL time, we need to use something like TMAEvent to combine multiple real events into a synthetic event.  See DM-46102.


In [10]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

,LSSTComCam,LSSTCam
Total Night,7:46:34,7:46:34
Total Exposure,7:52:36,25:14:34
Slew time(1),0:00:00,0:00:00
Readout time(2),0:43:53,4:00:18
Time loss to fault,NA,NA
Time loss to weather,NA,NA
Idle time,0:53:57,-17:31:59
Number of exposures,1145,6269
Mean readout time,0.000639,0.000639
Number of slews(1),0,0


- (1) *Expected* slew time as per Schedular
- (2) There is no practical way to get detector read-out time.  A value of 2.3 seconds was inferred from experiments.

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
(Observing Operations - OBS)
(TODO: better tickets using Jira API)

In [11]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

- [obs-321](https://rubinobs.atlassian.net/browse/obs-321)

- [/rubinobs-lfa-cp/LOVE/OLE/2024/11/04/LOVE_OLE_2024-11-05T04:06:17.057.png](https://s3.cp.lsst.org/rubinobs-lfa-cp/LOVE/OLE/2024/11/04/LOVE_OLE_2024-11-05T04:06:17.057.png)

- [OBS-643](https://rubinobs.atlassian.net/browse/OBS-643)

- [OBS-620](https://rubinobs.atlassian.net/browse/OBS-620)

- [OBS-627](https://rubinobs.atlassian.net/browse/OBS-627)

...

- [OBS-321](https://rubinobs.atlassian.net/browse/OBS-321)

- [obs-611](https://rubinobs.atlassian.net/browse/obs-611)

- [OBS-641](https://rubinobs.atlassian.net/browse/OBS-641)

- [/rubinobs-lfa-cp/LOVE/OLE/2024/11/03/LOVE_OLE_2024-11-03T21:41:04.106.png](https://s3.cp.lsst.org/rubinobs-lfa-cp/LOVE/OLE/2024/11/03/LOVE_OLE_2024-11-03T21:41:04.106.png)

- [obs-647](https://rubinobs.atlassian.net/browse/obs-647)

In [12]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

#### Nightly Jira BLOCKs
- 20241103
    - [testCycle/BLOCK-R126](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testCycle/BLOCK-R126)
- 20241102
    - [testCycle/BLOCK-R125](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testCycle/BLOCK-R125)

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
(TODO: BLOCKS observed) 

## Data Log <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

- NOTE: *modified all_sources.py:uniform_field_counts()*
- NOTE: Each tally count is now a link to a detailed table of exposures.

In [13]:
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    df_dict = allsrc.fields_count_exposure(instrum)
    for field_name,df in df_dict.items():
        print(field_name.title().replace('_',' '))
        display(df.T)

**LSSTComCamSim: 0 total exposures**

Observation Type


""


Observation Reason


""


...

,good,questionable,junk,unknown,total
unknown,0,0,0,1,1
dark,0,0,0,1784,1784
bias,0,0,0,1392,1392
flat,0,0,0,3092,3092


Observation Reason


,good,questionable,junk,unknown,total
sflat_hi,0,0,0,656,656
unknown,0,0,0,1,1
dark,0,0,0,776,776
bias,0,0,0,159,159
sflat_lo,0,0,0,10,10
flat,0,0,0,4634,4634
preimage,0,0,0,33,33


Science Program


,good,questionable,junk,unknown,total
E2180,0,0,0,3,3
E2234,0,0,0,1,1
E2013,0,0,0,33,33
E2229,0,0,0,24,24
E2014,0,0,0,14,14
E2235,0,0,0,1,1
E2016,0,0,0,4036,4036
E2015,0,0,0,23,23
E2010,0,0,0,4,4
E2230,0,0,0,201,201


In [14]:
# Exposure Report
if experimental.get("exposure_type_tally"): 
    #  md("### Experimental Exposure field Tally ENABLED")
    md("Tally exposures with specific values of selected fields")
else:
    md("### Experimental Exposure field Tally DISASBLED")
    
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    if 0 == len(allsrc.exp_src.exposures[instrum]):
        continue

    # Facet counts
    if experimental.get("exposure_type_tally"): 
        counts,cc,f = allsrc.uniform_exposure_field_counts(instrum)
        for fname, tally in []: # counts.items():
            md(f"{fname}")
            df = pd.DataFrame.from_dict(tally, orient="index").T
            wdf = ut.wrap_dataframe_columns(df)
            display(HTML(wdf.to_html(render_links=True, escape=False)))

### Experimental Exposure field Tally DISASBLED

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 1145 total exposures**

**LSSTCam: 6269 total exposures**

Note: *Added counts of Exposures with messages*

In [15]:
for instrum, recs in allsrc.exp_src.messages.items():
    md(f'**{instrum} {len(recs)} Exposures with messages**')
    if len(recs) > 0:
        df = allsrc.tally_exposure_flags(instrum)
        display(df.style.hide(axis="index"))

**LSSTComCam 31 Exposures with messages**

AttributeError: 'AllSources' object has no attribute 'tally_exposure_flags'

## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [ ]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px">REMOVE</font>
Contains stuff only expected to be useful to developers.

This may also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [ ]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
""")

## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [ ]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## DDV 

In [ ]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [ ]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

In [ ]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

### According to Storybaord, this section should be removed ...
... and replaced with a simple table tally of Good, Questionable, and  Junk exposures per instrument.
| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [ ]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

## Finale

In [ ]:
print(f"Finished {str(dt.datetime.now())}")